# SQL

## Описание проекта
Имеется база данных одного из сервисов, создающих приложения для чтения книг. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

C помощью языка запросов SQL необходимо получить ответы на ряд вопросов.

## Задачи
- Посчитать количество книг, выпущенных после 1 января 2000 года;
- Посчитать количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определить издательство, которое издало наибольшее число книг толще 50 страниц (так получится исключить из анализа различные брошюры);
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более пользовательскими оценками;
- Посчитать среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## 1. Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as psycopg2

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#Загрузка данных
query_books = ''' SELECT * FROM books'''
query_authors = ''' SELECT * FROM authors'''
query_publishers = ''' SELECT * FROM publishers'''
query_ratings = ''' SELECT * FROM ratings'''
query_reviews = ''' SELECT * FROM reviews'''

books=pd.io.sql.read_sql(query_books, con = engine)
authors=pd.io.sql.read_sql(query_authors, con = engine)
publishers=pd.io.sql.read_sql(query_publishers, con = engine)
ratings=pd.io.sql.read_sql(query_ratings, con = engine)
reviews=pd.io.sql.read_sql(query_reviews, con = engine)

## 2. Обзор данных 

### Данные о книгах

In [9]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [10]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


1000 книг, 6 параметров. 

### Данные об авторах

In [11]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [12]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


636 авторов, два параметра.

### Данные об издательствах

In [13]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [14]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


340 издательств, два параметра.

### Данные о пользовательских оценках книг

In [15]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [16]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


6456 оценок, 4 параметра.

### Данные о пользовательских обзорах на книги

In [17]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


2793 обзора, 4 параметра.

## 3. SQL-запросы

In [19]:
books.head(2)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


### Посчитайте количество книг, выпущенных после 1 января 2000 года

In [20]:
#количество книг, выпущенных после 1 января 2000 года

query = ''' 
SELECT 
    COUNT(*) AS cnt 
FROM 
    books 
WHERE 
    publication_date > '2000-01-01' 
'''

pd.io.sql.read_sql(query, con = engine)

,cnt
0,819


819 книг, выпущенных после 1 января 2000 г. 

### Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги

In [7]:
#количество пользовательских обзоров и средняя оценка для каждой книги

query = ''' 
SELECT 
    reviews.book_id, 
    books.title,
    reviews.cnt_reviews,
    ratings.avg_rating
    
FROM 
(SELECT 
    book_id, 
    COUNT(review_id) AS cnt_reviews
FROM
    reviews
--where book_id = 839    
GROUP BY 
    book_id
) AS reviews  
FULL OUTER JOIN 
(
SELECT 
    book_id, 
    AVG(rating) AS avg_rating 
FROM 
    ratings 
GROUP BY 
    book_id 
) AS ratings
ON reviews.book_id=ratings.book_id
FULL OUTER JOIN books ON books.book_id=ratings.book_id
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,cnt_reviews,avg_rating
0,652.0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,273.0,Galápagos,2.0,4.500000
2,51.0,A Tree Grows in Brooklyn,5.0,4.250000
3,951.0,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,839.0,The Prophet,4.0,4.285714
...,...,...,...,...
995,64.0,Alice in Wonderland,4.0,4.230769
996,55.0,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,148.0,Christine,3.0,3.428571
998,790.0,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


### Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры)

In [22]:
books.head(2)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


In [23]:
#издательство, которое издало наибольшее число книг толще 50 страниц

query = ''' SELECT 
    
    books.publisher_id AS publisher_id, 
    publishers.publisher AS publisher,
    COUNT(books.book_id) AS books_cnt    
FROM
    books
LEFT JOIN publishers ON books.publisher_id=publishers.publisher_id      
WHERE
    books.num_pages > 50
GROUP BY 
    books.publisher_id,
    publishers.publisher
ORDER BY 
    COUNT(books.book_id) DESC
LIMIT 1
    '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,books_cnt
0,212,Penguin Books,42


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками

In [24]:
#автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более пользовательскими оценками

query = ''' 
SELECT 
    authors.author_id as author_id,
    authors.author AS author,
    AVG(ratings.rating) AS avg_rating    
FROM
    (SELECT
        books.book_id,
        books.author_id,
        books.num_pages,
        COUNT (ratings.rating_id) AS cnt
    FROM
        books
    RIGHT JOIN ratings ON books.book_id=ratings.book_id
    GROUP BY 
        books.book_id,
        books.author_id,
        books.num_pages
    HAVING 
        COUNT (ratings.rating_id) > 50
    ) as books
LEFT JOIN authors ON books.author_id=authors.author_id 
LEFT JOIN ratings ON books.book_id=ratings.book_id
GROUP BY 
    authors.author,
    authors.author_id
ORDER BY 
    AVG(ratings.rating) DESC
LIMIT 1
    '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


### Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

In [25]:
#среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

query = ''' 
SELECT
    AVG (cnt_review) AS avg_revies
FROM
(
SELECT 
    reviews.username,
    COUNT(reviews.review_id) AS cnt_review    
FROM
    (SELECT
        username,
        COUNT (rating_id) AS cnt
    FROM
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT (rating_id) > 50
    ) AS users
    
LEFT JOIN reviews ON users.username=reviews.username 

GROUP BY 
    reviews.username
) AS review   
    '''

pd.io.sql.read_sql(query, con = engine)

,avg_revies
0,24.333333


## 4. Результаты


- Определено, что 819 книг выпущенно после 1 января 2000 г.
- Посчитано количество пользовательских обзоров и средняя оценка для каждой книги отдельно.
- Penguin Books - издательство, которое издало наибольшее число книг толще 50 страниц.
- J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более пользовательскими оценками.
- Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок, равно 24.